In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  2 10:39:59 2020

@author: c3121
"""


#%%
import os
import keras
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
def apply_color_mask(hsv,img,low,high):
    # Apply color mask to image
    mask = cv2.inRange(hsv, low, high)
    res = cv2.bitwise_and(img,img, mask= mask)
    return res

#%%
base_path = r'diease'#這裡要改
img_0=[]
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".JPG"):
            filename = os.path.join(root, file)
            file_size = os.path.getsize(filename)
            category_name = os.path.basename(root)
            if file_size >= 2048:
                #print(filename)
                im = cv2.imread(filename)
                #im = cv2.imread(r'diease/c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG')
                im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
                img_0.append(im)
img_0_array=np.asarray(img_0)
img_0_label=np.ones(img_0_array.shape[0])*0 
base_path = r'health'#這裡要改
img_1=[]
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".JPG"):
            filename = os.path.join(root, file)
            file_size = os.path.getsize(filename)
            category_name = os.path.basename(root)
            #print(filename)
            if file_size >= 2048:
                #print(filename)
                im = cv2.imread(filename)
                #print(filename)
                #im = cv2.imread(r'diease/c47ccf21-5249-4a95-8d33-0d640e3529f7___RS_L.Scorch 0158.JPG')
                im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
                img_1.append(im)
img_1_array=np.asarray(img_1)
img_1_label=np.ones(img_1_array.shape[0])*1
img_array=np.concatenate((img_0_array,img_1_array) ,axis=0)
img_label=np.concatenate((img_0_label,img_1_label),axis=0)
img_label = keras.utils.to_categorical(img_label, num_classes =2)
#print(img_label)
import random
temp_img = list(zip(img_array, img_label))
random.shuffle(temp_img)
img_array, img_label = zip(*temp_img)
img_array=np.asarray(img_array)
img_label=np.asarray(img_label)
print(img_label)
from sklearn.model_selection import train_test_split
img_train,img_test,label_train,label_test=train_test_split(img_array, img_label, test_size=0.2,shuffle = True)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [4]:
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, SGD
from keras.layers.normalization import BatchNormalization

model = Sequential()
    
    #First Convolution and Pooling layer
model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(128,128,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    
    #Second Convolution and Pooling layer
model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    
    #Three Convolution layer and Pooling Layer
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    
    #Fully connection layer
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))
    
    #Classfication layer
model.add(Dense(2,activation='softmax'))
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'

model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
print(len(img_train))
history=model.fit(img_train, label_train, batch_size=64, epochs=100,shuffle=True, validation_split=0.2)


2247
Epoch 1/100
29/29 [==============================] - 11s 390ms/step - loss: 4.2434 - accuracy: 0.5253 - val_loss: 0.6188 - val_accuracy: 0.4867
Epoch 2/100
29/29 [==============================] - 11s 384ms/step - loss: 0.7231 - accuracy: 0.6077 - val_loss: 0.6197 - val_accuracy: 0.6422
Epoch 3/100
29/29 [==============================] - 11s 377ms/step - loss: 0.5302 - accuracy: 0.7969 - val_loss: 3.7409 - val_accuracy: 0.5133
Epoch 4/100
29/29 [==============================] - 11s 383ms/step - loss: 0.4519 - accuracy: 0.8364 - val_loss: 0.4529 - val_accuracy: 0.7000
Epoch 5/100
29/29 [==============================] - 11s 372ms/step - loss: 0.2687 - accuracy: 0.8887 - val_loss: 0.5978 - val_accuracy: 0.6378
Epoch 6/100
29/29 [==============================] - 11s 380ms/step - loss: 0.4259 - accuracy: 0.8670 - val_loss: 0.4796 - val_accuracy: 0.7489
Epoch 7/100
29/29 [==============================] - 11s 389ms/step - loss: 0.2429 - accuracy: 0.9160 - val_loss: 0.1142 - val_accu

In [11]:
n=0
for index in range(len(img_test)):
    test_im=img_test[index]
   #plt.subplot(2,4,index+1)
    #plt.imshow(cv2.cvtColor(test_im, cv2.COLOR_BGR2RGB))
    test_im=test_im.reshape(-1,128,128,3)
    pred=model.predict(test_im)
    
    a=np.argmax(pred)
    b=np.argmax(label_test[index])
    if(a==b):
        n=n+1
    #plt.show()
print('準確:',n,'總共:' ,len(img_test))    

準確: 554 總共: 562
